In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [4]:
raw=pd.read_excel(r'data\aim_input.xlsx')

In [5]:
raw.columns

Index(['lat', 'lon', 'BRDF_Albedo_Parameters_Band1_iso',
       'BRDF_Albedo_Parameters_Band1_geo', 'BRDF_Albedo_Parameters_Band1_vol',
       'BRDF_Albedo_Parameters_Band2_iso', 'BRDF_Albedo_Parameters_Band2_geo',
       'BRDF_Albedo_Parameters_Band2_vol', 'BRDF_Albedo_Parameters_Band3_iso',
       'BRDF_Albedo_Parameters_Band3_geo', 'BRDF_Albedo_Parameters_Band3_vol',
       'BRDF_Albedo_Parameters_Band4_iso', 'BRDF_Albedo_Parameters_Band4_geo',
       'BRDF_Albedo_Parameters_Band4_vol', 'BRDF_Albedo_Parameters_Band5_iso',
       'BRDF_Albedo_Parameters_Band5_geo', 'BRDF_Albedo_Parameters_Band5_vol',
       'BRDF_Albedo_Parameters_Band6_iso', 'BRDF_Albedo_Parameters_Band6_geo',
       'BRDF_Albedo_Parameters_Band6_vol', 'BRDF_Albedo_Parameters_Band7_iso',
       'BRDF_Albedo_Parameters_Band7_geo', 'BRDF_Albedo_Parameters_Band7_vol',
       'BRDF_Albedo_Parameters_vis_iso', 'BRDF_Albedo_Parameters_vis_geo',
       'BRDF_Albedo_Parameters_vis_vol', 'BRDF_Albedo_Parameters_nir_iso',
   

In [6]:
raw[['GapPct_25_', 'GapPct_51_', 'GapPct_101',
       'GapPct_200', 'GapPct_251', 'Herbaceous_h', 'Woody_h', 'BareSoilCo',
       'TotalFolia', 'SageBrushC', 'SagebrushH', 'NonInvPereF', 'NonInvPeG',
       'NonInvAnnG', 'NonInvAnnF', 'NonInvShru', 'InvAnnGras', 'InvAnnForb']].head()

,GapPct_25_,GapPct_51_,GapPct_101,GapPct_200,GapPct_251,Herbaceous_h,Woody_h,BareSoilCo,TotalFolia,SageBrushC,SagebrushH,NonInvPereF,NonInvPeG,NonInvAnnG,NonInvAnnF,NonInvShru,InvAnnGras,InvAnnForb
0,2.633333,3.733333,7.266667,11.700000,25.333334,57.008888,27.751852,0.990099,56.435642,0.0,0.0,3.960396,36.633663,1.980198,0.000000,7.920792,0.0,0.0
1,2.966667,6.400000,1.100000,6.100000,16.566668,61.994816,60.301483,0.000000,64.356438,0.0,0.0,4.950495,40.594059,0.000000,1.980198,19.801981,0.0,0.0
2,1.033333,7.600000,19.066668,29.000000,56.700001,4.421482,29.868519,10.891089,35.643566,0.0,0.0,3.960396,0.000000,0.000000,0.000000,27.722773,0.0,0.0
3,0.400000,4.566667,16.933332,56.566666,78.466667,3.574815,31.373703,5.940594,18.811882,0.0,0.0,0.000000,0.000000,0.000000,0.000000,11.881188,0.0,0.0
4,12.400000,21.500000,31.500000,13.300000,78.300000,19.230000,0.500000,48.330000,39.330000,0.0,0.0,1.670000,27.670000,6.330000,1.670000,0.000000,0.0,0.0


In [7]:
scaled_raw=raw.copy()

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
dep=['temp90', 'prcp90', 'srad90', 'elevation', 'slope', 'aspect',
       'clay', 'sand', 'silt', 'GapPct_25_', 'GapPct_51_', 'GapPct_101',
       'GapPct_200', 'GapPct_251', 'Herbaceous_h', 'Woody_h', 'BareSoilCo',
       'TotalFolia', 'SageBrushC', 'SagebrushH', 'NonInvPereF', 'NonInvPeG',
       'NonInvAnnG', 'NonInvAnnF', 'NonInvShru', 'InvAnnGras', 'InvAnnForb']
std = StandardScaler()
scaled_raw[dep] = std.fit_transform(scaled_raw[dep])

In [9]:
indep_var=['GapPct_25_', 'GapPct_51_', 'GapPct_101',
       'GapPct_200', 'GapPct_251', 'Herbaceous_h', 'Woody_h', 'BareSoilCo',
       'TotalFolia', 'SageBrushC', 'SagebrushH', 'NonInvPereF', 'NonInvPeG',
       'NonInvAnnG', 'NonInvAnnF', 'NonInvShru', 'InvAnnGras', 'InvAnnForb']

In [10]:
df_in=scaled_raw.drop(['lat','lon'],axis=1)

In [11]:
from sklearn.ensemble import RandomForestRegressor     
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

In [12]:
pred2016=pd.read_csv(r'data\prediction\input\pred2016_input.txt')
pred2016.drop(['FID','Field19','Field20'],axis=1,inplace=True)
pred2016.head()

,b1_pred,b2_pred,b3_pred,b4_pred,b5_pred,b6_pred,b7_pred,b8_pred,b9_pred,b10_pred,...,b48_pred,b49_pred,b50_pred,b51_pred,b52_pred,b53_pred,b54_pred,b55_pred,b56_pred,b57_pred
0,0.110,0.067,0.037,0.178,0.145,0.049,0.055,0.043,0.015,0.087,...,0.053151,0.462542,0.027407,0.369504,0.059811,0.220580,0.391720,0.24,0.35,0.41
1,0.103,0.043,0.031,0.155,0.087,0.039,0.050,0.036,0.013,0.078,...,0.099155,0.451431,0.029630,0.485251,0.088722,0.308007,0.679113,0.35,0.31,0.34
2,0.104,0.013,0.035,0.147,0.064,0.038,0.056,0.000,0.019,0.081,...,0.099462,0.439226,0.030370,0.574195,0.115312,0.070556,1.655410,0.21,0.36,0.43
3,0.126,0.047,0.044,0.190,0.111,0.055,0.067,0.024,0.021,0.102,...,0.083692,0.470960,0.025926,0.387720,0.041840,0.087073,1.128040,0.21,0.39,0.40
4,0.096,0.069,0.030,0.150,0.101,0.035,0.048,0.049,0.012,0.075,...,0.050923,0.470034,0.029630,0.417860,0.084873,0.081243,1.190770,0.34,0.32,0.34


In [13]:
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14985 entries, 0 to 14984
Data columns (total 75 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   BRDF_Albedo_Parameters_Band1_iso      14985 non-null  float64
 1   BRDF_Albedo_Parameters_Band1_geo      14985 non-null  float64
 2   BRDF_Albedo_Parameters_Band1_vol      14985 non-null  float64
 3   BRDF_Albedo_Parameters_Band2_iso      14985 non-null  float64
 4   BRDF_Albedo_Parameters_Band2_geo      14985 non-null  float64
 5   BRDF_Albedo_Parameters_Band2_vol      14985 non-null  float64
 6   BRDF_Albedo_Parameters_Band3_iso      14985 non-null  float64
 7   BRDF_Albedo_Parameters_Band3_geo      14985 non-null  float64
 8   BRDF_Albedo_Parameters_Band3_vol      14985 non-null  float64
 9   BRDF_Albedo_Parameters_Band4_iso      14985 non-null  float64
 10  BRDF_Albedo_Parameters_Band4_geo      14985 non-null  float64
 11  BRDF_Albedo_Par

In [ ]:
#output pred for mapping and metrics
kf = KFold(n_splits=20)
count=0
for train,test in kf.split(df_in):
    r2=[]
    mse=[]
    rmse=[]
    fea_import=[]
    oob=[]
    print(train.shape)
    print(test.shape)
    count+=1
    X_train=df_in.loc[train,:]
    X_train.drop(indep_var,axis=1,inplace=True)
    X_test=df_in.loc[test,:]
    X_test.drop(indep_var,axis=1,inplace=True)

    for col in indep_var:
        Y_train=df_in.loc[train,col]      
        Y_test=df_in.loc[test,col]
        rf_reg=RandomForestRegressor(oob_score=True,random_state=0,n_jobs=-1)   
        rf_reg.fit(X_train,Y_train)
        pred_Y=rf_reg.predict(X_test)
        #prediction
        output=rf_reg.predict(pred2016)
        np.savetxt(r"pred2016/temp/ensemble_pred/"+str(col)+str(count)+'.txt',output,delimiter=',',fmt='%10.5f')
        #output metrics
        r2.append(r2_score(Y_test,pred_Y))
        mse.append(mean_squared_error(Y_test,pred_Y))
        rmse.append(sqrt(mean_squared_error(Y_test,pred_Y)))
        fea_import.append(rf_reg.feature_importances_)
        oob.append(rf_reg.oob_score_)
    np.savetxt(r'data/regression/metrics/ensemble/r_square'+str(count)+'.txt',r2,delimiter=',',fmt='%10.5f')
    np.savetxt(r'data/regression/metrics/ensemble/rmse'+str(count)+'.txt',rmse,delimiter=',',fmt='%10.5f')    
    np.savetxt(r'data/regression/metrics/ensemble/mse'+str(count)+'.txt',mse,delimiter=',',fmt='%10.5f')  
    np.savetxt(r'data/regression/metrics/ensemble/fea_import'+str(count)+'.txt',fea_import,delimiter=',',fmt='%10.5f')
    np.savetxt(r'data/regression/metrics/ensemble/oob'+str(count)+'.txt',oob,delimiter=',',fmt='%10.5f')

In [64]:
#output for scatter plot
kf = KFold(n_splits=20)
count=0
for train,test in kf.split(df_in):
    
    print(train.shape)
    print(test.shape)
    count+=1
    X_train=df_in.loc[train,:]
    X_train.drop(indep_var,axis=1,inplace=True)
    X_test=df_in.loc[test,:]
    X_test.drop(indep_var,axis=1,inplace=True)

    for col in indep_var:
        Y_train=df_in.loc[train,col]      
        Y_test=df_in.loc[test,col]
        rf_reg=RandomForestRegressor(oob_score=True,random_state=0,n_jobs=-1)   
        rf_reg.fit(X_train,Y_train)
        pred_Y=rf_reg.predict(X_test)
        #output pred vs actual for training
        pred_Y_train=rf_reg.predict(X_train)
        a=pred_Y_train.reshape(-1,1)
        b=Y_train.to_numpy().reshape(-1,1)
        output_train=np.append(a,b,1)
        np.savetxt(r"data/regression/pred_act/ensemble/"+str(col)+str(count)+'_train.txt',output_train,delimiter=',',fmt='%10.5f')
        #output pred vs test for testing
        c=pred_Y.reshape(-1,1)
        d=Y_test.to_numpy().reshape(-1,1)
        output_test=np.append(c,d,1)
        np.savetxt(r"data/regression/pred_act/ensemble/"+str(col)+str(count)+'_test.txt',output_test,delimiter=',',fmt='%10.5f')


(14235,)
(750,)
(14235,)
(750,)
(14235,)
(750,)
(14235,)
(750,)
(14235,)
(750,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
(14236,)
(749,)
